In [1]:
import os
import cv2
import numpy as np
from keras.models import load_model
import time
from playsound import playsound

# Suppress TensorFlow warnings for a cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print("Modules loaded successfully!")

Modules loaded successfully!


In [3]:
# Load pre-trained classifiers for face and eye detection using Haar cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load the pre-trained deep learning model for drowsiness detection
model = load_model('C:/Users/ROHIT KHETO/Desktop/4th_year/TrainedModel/Fold_0.h5')
print("Models and classifiers loaded successfully!")

Models and classifiers loaded successfully!


In [5]:
# Initialize variables for tracking drowsiness
drowsy_start_time = None
drowsy_duration_threshold = 2.5  # in seconds

# Corrected path for the alarm sound
alarm_audio_path = 'C:/Users/ROHIT KHETO/Desktop/4th_year/alarm-sound.mp3'

In [ ]:
# Start video capture from the default webcam
cap = cv2.VideoCapture(0)


while True:
    # Read the current frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Initialize variables for visualization and drowsiness status
    preprocessed_img = None
    detected_face_img = frame.copy()
    control_points_img = None
    drowsy_status = "Unknown"

    # Process detected faces
    if len(faces) > 0:
        # Select the largest detected face (assuming it's the primary subject)
        x, y, w, h = max(faces, key=lambda f: f[2] * f[3])
        face_region = gray[y:y+h, x:x+w]
        detected_face_img = frame[y:y+h, x:x+w].copy()

        # Detect eyes within the detected face region
        eyes = eye_cascade.detectMultiScale(face_region, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        if len(eyes) >= 2:
            # Find the bounding box covering both eyes
            min_x, max_x = min(eyes[0][0], eyes[1][0]), max(eyes[0][0] + eyes[0][2], eyes[1][0] + eyes[1][2])
            min_y, max_y = min(eyes[0][1], eyes[1][1]), max(eyes[0][1] + eyes[0][3], eyes[1][1] + eyes[1][3])
            eyes_region = gray[y + min_y:y + max_y, x + min_x:x + max_x]

            # Preprocess the eyes region and predict drowsiness
            if eyes_region.size > 0:
                preprocessed_img = cv2.resize(eyes_region, (150, 60)) / 255.0
                input_img = np.expand_dims(preprocessed_img, axis=(0, -1))
                predicted_class = np.argmax(model.predict(input_img), axis=1)[0]
                label_map = {0: "Not Drowsy", 1: "Drowsy"}
                drowsy_status = label_map.get(predicted_class, 'Unknown')

                # Visualize the detected eyes region
                control_points_img = frame.copy()
                cv2.rectangle(control_points_img, (x + min_x, y + min_y), (x + max_x, y + max_y), (0, 255, 0), 2)

    # Track the duration of drowsy status and play alarm if necessary
    if drowsy_status == "Drowsy":
        if drowsy_start_time is None:
            drowsy_start_time = time.time()
        elif time.time() - drowsy_start_time >= drowsy_duration_threshold:
            playsound(alarm_audio_path)  # Play the alarm sound
            drowsy_start_time = None  # Reset to avoid continuous alarm
    else:
        drowsy_start_time = None  # Reset if not drowsy

    # Display the current status on the video frame
    cv2.putText(frame, f"Status: {drowsy_status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the video frame with annotations
    cv2.imshow('Drowsiness Detection', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━